<a href="https://colab.research.google.com/github/guerrac2001/Colab-Notebooks/blob/main/DataYFinance_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Permitir a Colab trabajar desde drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalar Yahoo Finance para obtener los datos

In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 6.4 MB 23.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Obtener los precios desde una fecha de inicio hasta el día de hoy

In [ ]:
import yfinance as yahooFinance
import datetime
fechaInicio = datetime.datetime (2018, 1, 1)
fechaFin = datetime.datetime.now()
fechaInicioVal = datetime.datetime (2019,12,31)
dataM = yahooFinance.download("GOOG",start = fechaInicio, end=fechaFin, interval = "1mo", auto_adjust = True)
rangoValida = dataM.loc[fechaInicioVal:fechaFin].shape[0]

print (dataM)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close        Volume
Date                                                                    
2018-01-01   52.320000   59.344501   52.261501   58.497002  5.747697e+08
2018-02-01   58.130501   58.700001   49.627998   55.236500  8.476821e+08
2018-03-01   55.393501   58.852501   49.032001   51.589500  9.086010e+08
2018-04-01   51.140999   54.708248   49.518501   50.866501  8.354655e+08
2018-05-01   50.682999   55.537498   50.314499   54.249500  6.357471e+08
2018-06-01   54.967499   59.314281   54.800499   55.782501  6.420728e+08
2018-07-01   54.950001   63.694500   54.689999   60.862999  6.390677e+08
2018-08-01   61.400002   62.825001   59.411999   60.909500  5.764076e+08
2018-09-01   60.909500   60.909500   57.345501   59.673500  5.772640e+08
2018-10-01   59.994499   60.498001   49.791500   53.838501  9.699233e+08
2018-11-01   53.790001   54.778500   49.800999   54.721

Obtener los precios de cierre mensuales de la acción 

In [ ]:
PreciosCierre = dataM['Close'].values
print(PreciosCierre)

[ 58.49700165  55.23649979  51.58950043  50.86650085  54.24950027
  55.78250122  60.86299896  60.90950012  59.67350006  53.83850098
  54.7215004   51.7804985   55.81850052  55.99599838  58.66550064
  59.42399979  55.18149948  54.04550171  60.83399963  59.40499878
  60.95000076  63.00550079  65.2480011   66.85099792  71.71150208
  66.96649933  58.14049911  67.43299866  71.44599915  70.68049622
  74.14800262  81.70899963  73.48000336  81.05049896  88.03700256
  87.59400177  91.78700256 101.84300232 103.4315033  120.5059967
 120.57800293 125.31600189 135.22099304 145.46200562 133.26550293
 148.27049255 142.45199585 144.67950439 135.69850159 134.89100647
 139.64950562 114.96649933 114.03900146 109.37249756 112.76699829
          nan]


Instalar Torch

In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importar herramientas de Torch, Numpy y Matplotlib

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import (Dataset, TensorDataset,
 DataLoader, Subset)
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import os
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
NumCiclos = 2000
NumRezago = 3
BATCH_SIZE= 5

Definr métodos para generar Tensores y Dataset y generador aleatorio

Creación de datos en arreglos / convertir en tensores

In [ ]:
def crearDatosEntrada(series, nAtras=1):
    '''
    Transformar datos para CNN.
    
    Parametros
    ----------
    series : np.array
        La serie de tiempo para ser transformada
    nAtras : int
        numero de observaciones hacia atrás
        
    Devuelve
    -------
    x : np.array
        Arreglo precio
    y : np.array
        Arreglo de pronostico
    '''
    x = []
    y = []
    for precio in range(len(series) - nAtras):
        finPrecio = precio + nAtras
        x.append(series[precio:finPrecio])
        y.append(series[finPrecio])
    return np.array(x), np.array(y)
    
x, y= crearDatosEntrada (PreciosCierre, NumRezago)

xTensor = torch.from_numpy(x).float()
yTensor = torch.from_numpy(y).float().unsqueeze(dim=1)

print (xTensor)
print (xTensor.shape)
print (yTensor)
print (yTensor.shape)
plt.plot(xTensor)
plt.plot(yTensor)
plt.show()

Crear dataset para aprendisaje y entrenamiento de red

In [ ]:
def generaAleatoreaSemi(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:

generaAleatoreaSemi(42)

rangoBase = len(x) - rangoValida

X_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float().unsqueeze(dim=1)


dataset = TensorDataset(xTensor, yTensor)
rangoBase = len (x) - rangoValida

entrenarDataset = Subset(dataset, list(range(rangoBase)))
validarDataset = Subset(dataset, list(range(rangoBase, len(x))))

cargaEntreno = DataLoader(dataset=entrenarDataset,
                          batch_size=BATCH_SIZE)
cargaValidar = DataLoader(dataset=validarDataset, 
                          batch_size=BATCH_SIZE)


Definir arquitectura CNN

In [ ]:
class Flatten (nn.Module):
 def forward(self, X):
  return x.view(x.size()[0], -1)
 
model = nn.Sequential(OrderedDict([
 ('conv_1', nn.Conv1d(1, 32, 3, padding=1)),
 ('max_pool_1', nn.MaxPool1d(2)),
 ('relu_1', nn.ReLU()),
 ('flatten', Flatten()),
 ('fc_1', nn.Linear(192, 50)),
 ('relu_2', nn.ReLU()),
 ('dropout_1', nn.Dropout(0.4)),
 ('fc_2', nn.Linear(50, 1))
]))
print(model)


Sequential(
  (conv_1): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (max_pool_1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu_1): ReLU()
  (flatten): Flatten()
  (fc_1): Linear(in_features=192, out_features=50, bias=True)
  (relu_2): ReLU()
  (dropout_1): Dropout(p=0.4, inplace=False)
  (fc_2): Linear(in_features=50, out_features=1, bias=True)
)


Instanciando el modelo, funcion de perdida y optimizador

In [ ]:
model = model.to(device)
lossFn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Entrenado la CNN

In [ ]:
ImprimirCada = 50
entrenarPerdida, validarPerdida = [], []

for epoch in range(NumCiclos):
    ejecutarEntrenarPerdida = 0
    ejecutarValidarPerdida = 0

    model.train()
    
    for x_batch, y_batch in cargaEntreno:
        
        optimizer.zero_grad()
        
        x_batch = x_batch.to(device)
        x_batch = x_batch.view(x_batch.shape[0], 1, NumRezago)
        y_batch = y_batch.to(device)
        y_batch = y_batch.view(y_batch.shape[0], 1, 1)
        y_hat = model(x_batch).view(y_batch.shape[0], 1, 1)
        loss = torch.sqrt(lossFn(y_batch, y_hat))
        loss.backward()
        optimizer.step()
        ejecutarEntrenarPerdida += loss.item() * x_batch.size(0)
        
    epoch_loss_train = ejecutarEntrenarPerdida / len(cargaEntreno.dataset)
    entrenarPerdida.append(epoch_loss_train)

    with torch.no_grad():
        model.eval()
        for x_val, y_val in cargaValidar:
            x_val = x_val.to(device)
            x_val = x_val.view(x_val.shape[0], 1, NumRezago)
            y_val = y_val.to(device)
            y_val = y_val.view(y_val.shape[0], 1, 1)
            y_hat = model(x_val).view(y_val.shape[0], 1, 1)
            loss = torch.sqrt(lossFn(y_val, y_hat))
            ejecutarValidarPerdida += loss.item() * x_val.size(0)
            
        epoch_loss_valid = ejecutarValidarPerdida / len(cargaValidar.dataset)
            
        if epoch > 0 and epoch_loss_valid < min(validarPerdida):
            best_epoch = epoch
            torch.save(model.state_dict(), './cnn_checkpoint.pth')
            
        validarPerdida.append(epoch_loss_valid)

    if epoch % ImprimirCada == 0:
        print(f"<{epoch}> - Train. loss: {epoch_loss_train:.6f} \t Valid. loss: {epoch_loss_valid:.6f}")
        
print(f'Menor error encontrado en los ciclos: {best_epoch}')

TypeError: ignored